## As i have used some random dataset from kaggle and this is the approach i have learned from multiple medium blog posts but this might not be the best approach to solve this problem. I am still learning and trying to use best possible pipeline. NOTE: FAISS requires a lot of time and memory so i will update this code Friday afternoon as well so just read but don't run this code.

#### Run below cell only if you have not installed sentence-transformers library

In [1]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


#### Importing the necessary libraries

In [7]:
import pandas as pd
import numpy as np

#### Reading the data

In [3]:
df = pd.read_csv("wiki_movie_plots_deduped.csv")

#### First 5 rows of the data

In [4]:
df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


#### Checking shape of the data

In [9]:
df.shape

(34886, 9)

Checking null values in the data

In [11]:
df.isnull().sum()

Release Year           0
Title                  0
Origin/Ethnicity       0
Director               0
Cast                1422
Genre                  0
Wiki Page              0
Plot                   0
doc_len                0
dtype: int64

#### Dropping null values 

In [12]:
df.dropna(inplace=True)

#### Checking shape of the data after dropping null values

In [14]:
df.shape

(33464, 9)

#### Importing sentence-transformers and pre-trained model

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

#### Encoding Plot Data and Creating a FAISS Index

In [ ]:
import faiss
encoded_data = model.encode(df.Plot.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'movie_plot.index')

#### Function fetch the title and the first 500 characters of the plot

In [ ]:
def fetch_movie_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = dict()
    meta_dict['Title'] = info['Title']
    meta_dict['Plot'] = info['Plot'][:500]
    return meta_dict

#### Performs semantic search and returns top-k movie results based on the query

In [ ]:
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [ ]:
from pprint import pprint
query="Action movies: "
results=search(query, top_k=2, index=index, model=model)
print("\n")
for result in results:
    print('\t',result)

In [ ]:
query="Movies about romance and pain: "
results=search(query, top_k=5, index=index, model=model)

print("\n")
for result in results:
    print('\t',result)